## Lab 3 COMP 6315
## Edgar Acuna
## Imputation II
### Imputacion por la media media/mediana y moda y usando knn en los datasets: Titanic, Breastw y Segment 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing 

In [2]:
#Ejemplo1. Leyendo los datos de Titanic
titanic=pd.read_csv('http://academic.uprm.edu/eacuna/titanic.csv',header=0,sep=',',na_values='')
titanic.info()
titanic.head(39)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


PassengerId  Pclass                                               Name  \
0             1       3                            Braund, Mr. Owen Harris   
1             2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2             3       3                             Heikkinen, Miss. Laina   
3             4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4             5       3                           Allen, Mr. William Henry   
5             6       3                                   Moran, Mr. James   
6             7       1                            McCarthy, Mr. Timothy J   
7             8       3                     Palsson, Master. Gosta Leonard   
8             9       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9            10       2                Nasser, Mrs. Nicholas (Adele Achem)   
10           11       3                    Sandstrom, Miss. Marguerite Rut   
11           12       1                           Bonnell, Miss. Elizabeth   
12           13       3                     Saundercock, Mr. William Henry   
13           14       3                        Andersson, Mr. Anders Johan   
14           15       3               Vestrom, Miss. Hulda Amanda Adolfina   
15           16       2                   Hewlett, Mrs. (Mary D Kingcome)    
16           17       3                               Rice, Master. Eugene   
17           18       2                       Williams, Mr. Charles Eugene   
18           19       3  Vander Planke, Mrs. Julius (Emelia Maria Vande...   
19           20       3                            Masselmani, Mrs. Fatima   
20           21       2                               Fynney, Mr. Joseph J   
21           22       2                              Beesley, Mr. Lawrence   
22           23       3                        McGowan, Miss. Anna "Annie"   
23           24       1                       Sloper, Mr. William Thompson   
24           25       3                      Palsson, Miss. Torborg Danira   
25           26       3  Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...   
26           27       3                            Emir, Mr. Farred Chehab   
27           28       1                     Fortune, Mr. Charles Alexander   
28           29       3                      O'Dwyer, Miss. Ellen "Nellie"   
29           30       3                                Todoroff, Mr. Lalio   
30           31       1                           Uruchurtu, Don. Manuel E   
31           32       1     Spencer, Mrs. William Augustus (Marie Eugenie)   
32           33       3                           Glynn, Miss. Mary Agatha   
33           34       2                              Wheadon, Mr. Edward H   
34           35       1                            Meyer, Mr. Edgar Joseph   
35           36       1                     Holverson, Mr. Alexander Oskar   
36           37       3                                   Mamee, Mr. Hanna   
37           38       3                           Cann, Mr. Ernest Charles   
38           39       3                 Vander Planke, Miss. Augusta Maria   

       Sex   Age  SibSp  Parch            Ticket      Fare        Cabin  \
0     male  22.0      1      0         A/5 21171    7.2500          NaN   
1   female  38.0      1      0          PC 17599   71.2833          C85   
2   female  26.0      0      0  STON/O2. 3101282    7.9250          NaN   
3   female  35.0      1      0            113803   53.1000         C123   
4     male  35.0      0      0            373450    8.0500          NaN   
5     male   NaN      0      0            330877    8.4583          NaN   
6     male  54.0      0      0             17463   51.8625          E46   
7     male   2.0      3      1            349909   21.0750          NaN   
8   female  27.0      0      2            347742   11.1333          NaN   
9   female  14.0      1      0            237736   30.0708          NaN   
10  female   4.0      1      1           PP 9549   16.7000

In [3]:
#Eiminando las columnas PassengerId, Name y cabin(la cual tiene demasiados
#missing values)
titanic1=titanic.iloc[:,[1,3,4,5,6,7,8,10]]
titanic1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
Pclass      1309 non-null int64
Sex         1309 non-null object
Age         1046 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null object
Fare        1308 non-null float64
Embarked    1307 non-null object
dtypes: float64(2), int64(3), object(3)
memory usage: 81.9+ KB


In [4]:
#Imputando los missing values por la media si el atributo es continuo
#y por la moda si el atributo es nominal
from sklearn.preprocessing import Imputer
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
X=titanic1
xt = DataFrameImputer().fit_transform(X)

In [5]:
xt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
Pclass      1309 non-null int64
Sex         1309 non-null object
Age         1309 non-null float64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null object
Fare        1309 non-null float64
Embarked    1309 non-null object
dtypes: float64(2), int64(3), object(3)
memory usage: 81.9+ KB


In [6]:
#Imputando los missing values por la mediana si el atributo es continuo
#y por la moda si el atributo es nominal

class DataFrameImputer(TransformerMixin):

    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
X=titanic1
xt = DataFrameImputer().fit_transform(X)

### IMPUTACION k-nn

In [7]:
#imputando por knn
from sklearn import datasets
from sklearn import neighbors

class ImputeLearn(object):
    def __init__(self, mat):
        self.mat = np.array(mat)
        self.mat_isnan = np.isnan(self.mat)
        self.w = np.where(np.isnan(self.mat))

    def impute(self, learner):
        ximp = self.mat.copy()
        for i in np.arange(0, len(self.w[0])):
            n = self.w[0][i] # row where the nan is
            p = self.w[1][i] # column where the nan is
            col_isnan = self.mat_isnan[n, :] # empty columns in row n
            train = np.delete(self.mat, n, axis = 0) # remove row n to obtain a training set
            train_nonan = train[~np.apply_along_axis(np.any, 1, np.isnan(train)), :] # remove rows where there is a nan in the training set
            target = train_nonan[:, p] # vector to be predicted
            feature = train_nonan[:, ~col_isnan] # matrix of predictors
            learn = learner.fit(feature, target) # learner
            ximp[n, p] = learn.predict(self.mat[n, ~col_isnan].reshape(1, -1)) # predict and replace
        
        return(ximp)

Ejemplo 2 Aplicacion de imputacion k-nn al conjunto de datos Breast-wisconsin

In [8]:
breastdf=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data",header=None, sep=",",na_values=['?'])

In [9]:
breastdf.columns=['idx','v1','v2','v3','v4','v5','v6','v7','v8','v9','class']

In [10]:
breastdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
idx      699 non-null int64
v1       699 non-null int64
v2       699 non-null int64
v3       699 non-null int64
v4       699 non-null int64
v5       699 non-null int64
v6       683 non-null float64
v7       699 non-null int64
v8       699 non-null int64
v9       699 non-null int64
class    699 non-null int64
dtypes: float64(1), int64(10)
memory usage: 60.1 KB


In [11]:
breastdf.head()

idx  v1  v2  v3  v4  v5    v6  v7  v8  v9  class
0  1000025   5   1   1   1   2   1.0   3   1   1      2
1  1002945   5   4   4   5   7  10.0   3   2   1      2
2  1015425   3   1   1   1   2   2.0   3   1   1      2
3  1016277   6   8   8   1   3   4.0   3   7   1      2
4  1017023   4   1   1   3   2   1.0   3   1   1      2

In [12]:
breast1=breastdf.iloc[:,1:11]

In [13]:
breast1.head()

v1  v2  v3  v4  v5    v6  v7  v8  v9  class
0   5   1   1   1   2   1.0   3   1   1      2
1   5   4   4   5   7  10.0   3   2   1      2
2   3   1   1   1   2   2.0   3   1   1      2
3   6   8   8   1   3   4.0   3   7   1      2
4   4   1   1   3   2   1.0   3   1   1      2

In [14]:
b1=np.array(breast1)

In [15]:
data_imp = ImputeLearn(b1).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [16]:
completo5=pd.DataFrame(data_imp)

In [17]:
completo5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
0    699 non-null float64
1    699 non-null float64
2    699 non-null float64
3    699 non-null float64
4    699 non-null float64
5    699 non-null float64
6    699 non-null float64
7    699 non-null float64
8    699 non-null float64
9    699 non-null float64
dtypes: float64(10)
memory usage: 54.7 KB


Ejemplo. Imputacion k-nn a Titanic. Hay que convertir las variables categoticas a numericas porque knn-imputacion olo esta implementada para variables numericas 

In [18]:
#Convirtiendo las variables nominales en numericas, puesto que 
#la imputacion knn de python no trabaja con variables mezcladas
titanic2=titanic1.apply(preprocessing.LabelEncoder().fit_transform)

C:\Users\edgar2017\Anaconda2\envs\ipykernel_py2\lib\site-packages\numpy\lib\arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [19]:
titanic2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
Pclass      1309 non-null int64
Sex         1309 non-null int64
Age         1309 non-null int64
SibSp       1309 non-null int64
Parch       1309 non-null int64
Ticket      1309 non-null int64
Fare        1309 non-null int64
Embarked    1309 non-null int64
dtypes: int64(8)
memory usage: 81.9 KB


In [20]:
titanic2.head(10)

Pclass  Sex  Age  SibSp  Parch  Ticket  Fare  Embarked
0       2    1   32      1      0     720    20         3
1       0    0   57      1      0     816   237         1
2       2    0   39      0      0     914    50         3
3       0    0   53      1      0      65   217         3
4       2    1   53      0      0     649    52         3
5       2    1  134      0      0     373    60         2
6       0    1   76      0      0     109   214         3
7       2    1    8      3      1     541   143         3
8       2    0   41      0      2     477    87         3
9       1    0   21      1      0     174   177         1

In [21]:
b1=np.array(titanic2)

In [22]:
titanic_knnimp=ImputeLearn(b1).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [23]:
completo=pd.DataFrame(titanic_knnimp)
completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
0    1309 non-null int64
1    1309 non-null int64
2    1309 non-null int64
3    1309 non-null int64
4    1309 non-null int64
5    1309 non-null int64
6    1309 non-null int64
7    1309 non-null int64
dtypes: int64(8)
memory usage: 81.9 KB


In [24]:
#Ejemplo 2. Simulando misising values e imputandolos wn wl dataset segment de la UCI
seg=pd.read_table('http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/segment/segment.dat',header=None, delim_whitespace=True)
seg.columns=['v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','class']

In [25]:
seg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       2310 non-null float64
v2       2310 non-null float64
v3       2310 non-null int64
v4       2310 non-null float64
v5       2310 non-null float64
v6       2310 non-null float64
v7       2310 non-null float64
v8       2310 non-null float64
v9       2310 non-null float64
v10      2310 non-null float64
v11      2310 non-null float64
v12      2310 non-null float64
v13      2310 non-null float64
v14      2310 non-null float64
v15      2310 non-null float64
v16      2310 non-null float64
v17      2310 non-null float64
v18      2310 non-null float64
v19      2310 non-null float64
class    2310 non-null int64
dtypes: float64(18), int64(2)
memory usage: 361.0 KB


Todos las variables de segment son cuantitativas.

In [26]:
#simulando un 5% de celdas missing
import random
seg5=seg.copy()
seg5= seg5.stack().sample(frac=0.95).unstack().reindex(index=seg5.index, columns=seg5.columns)


In [27]:
seg5.head()

v1     v2   v3        v4   v5        v6        v7        v8        v9  \
0  218.0  178.0  9.0  0.111111  0.0  0.833333  0.547722  1.111109  0.544331   
1  113.0  130.0  9.0  0.000000  0.0  0.277778  0.250924  0.333333  0.365148   
2  202.0   41.0  9.0  0.000000  0.0  0.944448  0.772202  1.111112  1.025597   
3   32.0  173.0  9.0  0.000000  0.0  1.722222  1.781593  9.000000  6.749488   
4   61.0  197.0  9.0  0.000000  0.0  1.444444  1.515353  2.611111  1.925463   

          v10         v11         v12         v13        v14       v15  \
0   59.629630         NaN   75.222220   51.222220 -21.555555  46.77778   
1    0.888889         NaN    2.555556    0.111111  -2.666667   5.00000   
2  123.037040  111.888885  139.777790  117.444440 -33.444443  50.22222   
3   43.592594   39.555557   52.888890   38.333336 -12.111111  27.88889   
4   49.592594   44.222220   61.555557   43.000000 -16.111110  35.88889   

         v16         v17       v18       v19  class  
0 -25.222221   75.222220  0.318996 -2.040554    6.0  
1  -2.333333    2.555556  1.000000 -2.123254    3.0  
2 -16.777779  139.777790  0.199347 -2.299918    2.0  
3 -15.777778         NaN       NaN -1.998858    6.0  
4 -19.777779   61.555557  0.302925 -2.022274    6.0

In [28]:
seg5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       2196 non-null float64
v2       2209 non-null float64
v3       2183 non-null float64
v4       2200 non-null float64
v5       2205 non-null float64
v6       2209 non-null float64
v7       2195 non-null float64
v8       2186 non-null float64
v9       2190 non-null float64
v10      2191 non-null float64
v11      2184 non-null float64
v12      2175 non-null float64
v13      2205 non-null float64
v14      2203 non-null float64
v15      2187 non-null float64
v16      2191 non-null float64
v17      2195 non-null float64
v18      2187 non-null float64
v19      2198 non-null float64
class    2201 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [29]:
#imputando por la media
seg5.mean=seg5.apply(lambda x: x.fillna(x.mean()),axis=0)

In [30]:
#Imputando por la mediana
seg5.median=seg5.apply(lambda x: x.fillna(x.median()),axis=0)

In [31]:
#imputando usando knn
from sklearn import datasets
from sklearn import neighbors

class ImputeLearn(object):
    def __init__(self, mat):
        self.mat = np.array(mat)
        self.mat_isnan = np.isnan(self.mat)
        self.w = np.where(np.isnan(self.mat))

    def impute(self, learner):
        ximp = self.mat.copy()
        for i in np.arange(0, len(self.w[0])):
            n = self.w[0][i] # row where the nan is
            p = self.w[1][i] # column where the nan is
            col_isnan = self.mat_isnan[n, :] # empty columns in row n
            train = np.delete(self.mat, n, axis = 0) # remove row n to obtain a training set
            train_nonan = train[~np.apply_along_axis(np.any, 1, np.isnan(train)), :] # remove rows where there is a nan in the training set
            target = train_nonan[:, p] # vector to be predicted
            feature = train_nonan[:, ~col_isnan] # matrix of predictors
            learn = learner.fit(feature, target) # learner
            ximp[n, p] = learn.predict(self.mat[n, ~col_isnan].reshape(1, -1)) # predict and replace
        
        return(ximp)

In [32]:
b1=np.array(seg5)

In [33]:
data_imp = ImputeLearn(b1).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [34]:
completoknn5=pd.DataFrame(data_imp)

In [35]:
completoknn5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
0     2310 non-null float64
1     2310 non-null float64
2     2310 non-null float64
3     2310 non-null float64
4     2310 non-null float64
5     2310 non-null float64
6     2310 non-null float64
7     2310 non-null float64
8     2310 non-null float64
9     2310 non-null float64
10    2310 non-null float64
11    2310 non-null float64
12    2310 non-null float64
13    2310 non-null float64
14    2310 non-null float64
15    2310 non-null float64
16    2310 non-null float64
17    2310 non-null float64
18    2310 non-null float64
19    2310 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [36]:
from sklearn.metrics import mean_squared_error
#Calculando el cuadrado medio del error para imputacion por la media
completo5mean=pd.DataFrame(seg5.mean)
mean_squared_error(seg,completo5mean)

45.084197361638076

In [37]:
#Calculando el cuadrado medio del error para imputacion por la mediana
completo5median=pd.DataFrame(seg5.median)
mean_squared_error(seg,completo5median)

48.415462150527915

In [38]:
#Calculando el cuadrado medio del error para imputacion knn
mean_squared_error(seg,completoknn5)

17.080243475747896

Notar que el mejor metodo de imputacion ha sido knn

In [39]:
#simulando un 20% de celdas missing
seg20=seg.copy()
seg20= seg20.stack().sample(frac=0.80).unstack().reindex(index=seg20.index, columns=seg20.columns)
seg20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 20 columns):
v1       1831 non-null float64
v2       1880 non-null float64
v3       1853 non-null float64
v4       1838 non-null float64
v5       1848 non-null float64
v6       1838 non-null float64
v7       1861 non-null float64
v8       1839 non-null float64
v9       1850 non-null float64
v10      1851 non-null float64
v11      1843 non-null float64
v12      1856 non-null float64
v13      1858 non-null float64
v14      1833 non-null float64
v15      1807 non-null float64
v16      1850 non-null float64
v17      1852 non-null float64
v18      1862 non-null float64
v19      1844 non-null float64
class    1866 non-null float64
dtypes: float64(20)
memory usage: 361.0 KB


In [40]:
#hallando el numero de filas que tienen missings
sum(seg20.apply(lambda x: sum(x.isnull().values), axis = 1)>0)

2274

In [41]:
#imputando por la media al 20%
seg20.mean=seg20.apply(lambda x: x.fillna(x.mean()),axis=0)
completo20mean=pd.DataFrame(seg20.mean)
mean_squared_error(seg,completo20mean)

204.35096335537494

In [42]:
#imputando por la mediana al 20%
seg20.median=seg20.apply(lambda x: x.fillna(x.median()),axis=0)
completo20median=pd.DataFrame(seg20.median)
mean_squared_error(seg,completo20median)

216.40992988263866

In [43]:
b2=np.array(seg20)

In [44]:
data_imp = ImputeLearn(b2).impute(learner = neighbors.KNeighborsRegressor(n_neighbors = 5))

In [45]:
completoknn20=pd.DataFrame(data_imp)
mean_squared_error(seg,completoknn20)

131.07975256339515

Nuevamente imputacion por knn resulto ser la mejor